# Análisis Descriptivo ENO

### 1. Introducción

Este notebook presenta un análisis descriptivo y visual de los datos sobre las enfermedades de notificación obligatoria (ENO) en Chile. Exploraremos la distribución de las enfermedades, su evolución temporal, y otros factores relevantes.

Descripción del conjunto de datos: El Departamento de Epidemiología del Ministerio de Salud de Chile pone a disposición pública, archivo con bases de datos de algunas Enfermedades de Notificación Obligatoria (ENO) que se encuentran bajo vigilancia epidemiológica según lo establece el Decreto Supremo N°7/2019 MINSAL. Los datos corresponden al periodo 2007- 2022. Las ENO en la base de datos son: Arbovirus, Botulismo Adulto, Botulismo Infantil, Brucelosis, Carbunco, Chagas, Cisticercosis, Coqueluche, Creutzfeldt-Jakob, Difteria, Enfermedad meningocócica, Fiebre Tifoidea y Paratifoidea, Fiebre Q, Hepatitis A, Hepatitis B, Hepatitis C, Hepatitis E, Hidatidosis (Equinococosis), Infección gonocócica, Lepra, Leptospirosis, Malaria, Meningitis Bacteriana, MPOX, Parotiditis, Psitacosis, Sarampión, Sífilis, Síndrome Pulmonar por Hantavirus, Tétanos, Triquinosis, VIH.

Consideraciones:

- Los años 2019 al 2022 están sujetos a validación, por lo que podrían ser modificados en la siguiente actualización de datos. 
- Existen algunos campos que se encuentran vacíos desde el 2007-2018 debido a cambios realizados en los sistemas de información de notificación.

### 2. Carga y Exploración de Datos

In [1]:
# Librerías
import pandas as pd
import numpy as np
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt
from scipy import stats

# inhabilita warnings
import warnings
warnings.filterwarnings("ignore")

In [5]:
df = pd.read_csv('C:/Users/aalei/Desktop/github_dq/Datos/ENO Notification/eno_cleaned.csv', index_col=0)
print('Rows: ', df.shape[0], '  Columns: ', df.shape[1])
df.head(2)

Rows:  294352   Columns:  17


,etapa_clinica,region,seremi,nacionalidad,cie_10_diagnostico,diagnostico,anho_notificacion,ENO,pueblo_indigena,nombre_instruccion,existe_coinfeccion,via_transmision,uso_preservativo,pais_contagio,sexo,grupo_edad,codigo_comuna_residencia
0,CONFIRMADA,Región de Tarapacá,TARAPACÁ,Extranjero,A97.0,Dengue sin datos de alarma,2020,Arbovirus,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,mujer,25-29,01107
1,CONFIRMADA,Región de Valparaíso,VALPARAÍSO,Desconocido,A90.X,Fiebre del dengue,2018,Arbovirus,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,hombre,70-74,05301


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 294352 entries, 0 to 294351
Data columns (total 17 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   etapa_clinica             294352 non-null  object
 1   region                    294352 non-null  object
 2   seremi                    294352 non-null  object
 3   nacionalidad              294352 non-null  object
 4   cie_10_diagnostico        294352 non-null  object
 5   diagnostico               294352 non-null  object
 6   anho_notificacion         294352 non-null  int64 
 7   ENO                       294352 non-null  object
 8   pueblo_indigena           294352 non-null  object
 9   nombre_instruccion        294352 non-null  object
 10  existe_coinfeccion        294352 non-null  object
 11  via_transmision           294352 non-null  object
 12  uso_preservativo          294352 non-null  object
 13  pais_contagio             294352 non-null  object
 14  sexo     

In [5]:
# Datos int
df.describe().T

,count,mean,std,min,25%,50%,75%,max
anho_notificacion,294352.0,2015.718116,4.321443,2007.0,2012.0,2017.0,2019.0,2022.0
anio_aprobado,57001.0,2.980176,1.981536,1.0,1.0,3.0,4.0,10.0
n_parejas_sexuales,66841.0,3.021005,18.004775,0.0,1.0,1.0,2.0,1000.0
edad,17990.0,23.692385,14.361373,0.0,14.0,22.0,30.0,96.0
recuento_linfocitos,42617.0,344.081165,1837.719131,-29.0,102.0,279.0,461.0,230420.0


In [31]:
#Unique para observar erratas en literales
for i in df:
    print('VARIABLE: ' + i)
    print(df[i].unique())
    print(' --- ')

VARIABLE: etapa_clinica
['CONFIRMADA' 'PROBABLE']
 --- 
VARIABLE: region
['Región de Tarapacá' 'Región de Valparaíso'
 'Región Metropolitana de Santiago' 'Región de Los Lagos'
 'Región de Arica y Parinacota' 'Región del Biobío' 'Región de Los Ríos'
 'Región de Antofagasta' 'Región de la Araucanía'
 'Región del Libertador General Bernardo OHiggins'
 'Región de Aysén del General Carlos Ibáñez del Campo' 'Región del Ñuble'
 'Región del Maule' 'Región de Atacama' 'Región de Coquimbo'
 'Región de Magallanes y la Antártica Chilena']
 --- 
VARIABLE: seremi
['TARAPACÁ' 'VALPARAÍSO' 'RM' 'LOS LAGOS' 'ARICA Y PARINACOTA' 'BIOBÍO'
 'LOS RÍOS' 'ANTOFAGASTA' 'ARAUCANÍA' 'OHIGGINS' 'AYSÉN' 'ÑUBLE' 'MAULE'
 'ATACAMA' 'COQUIMBO' 'MAGALLANES' 'BIOBIO']
 --- 
VARIABLE: nacionalidad
['Extranjero' 'Desconocido' 'Chile']
 --- 
VARIABLE: cie_10_diagnostico
['A97.0' 'A90.X' 'A97.9' 'A90' 'A97.2' 'A97.1' 'U06.9' 'A92.0' 'A95.9'
 'A23.1' 'A23.9' 'A23.3' 'A23.8' 'A23.0' 'A23' 'B69.0' 'A37.0' 'A37.1'
 'A37.9' 'A

### 3. Preprocessing

**'region'**: reemplazar valor 'Desconocido' por la región, de acuerdo al valor de la variable 'codigo_comuna_residencia'

In [8]:
# Función para obtener la moda, ignorando los valores 'Desconocido'

def moda_sin_desconocido(grupo):
    return grupo[grupo != 'Desconocido'].mode()[0] if not grupo[grupo != 'Desconocido'].mode().empty else 'Desconocido'

# Calcular la moda de 'region' agrupada por 'codigo_comuna_residencia'
modas_por_comuna = df.groupby('codigo_comuna_residencia')['region'].transform(moda_sin_desconocido)

# Reemplazar 'Desconocido' por la moda correspondiente
df['region'] = df.apply(lambda row: modas_por_comuna[row.name] if row['region'] == 'Desconocido' else row['region'], axis=1)


**'seremi'**: Reemplazar los valores nulos de acuerdo con los valores de 'region'

In [26]:
# Diccionario de mapeo de región a SEREMI
region_to_seremi = {
    'Región de Tarapacá': 'TARAPACÁ',
    'Región de Valparaíso': 'VALPARAÍSO',
    'Región Metropolitana de Santiago': 'RM',
    'Región de Los Lagos': 'LOS LAGOS',
    'Región de Arica y Parinacota': 'ARICA Y PARINACOTA',
    'Región del Biobío': 'BIOBIO',
    'Región de Los Ríos': 'LOS RÍOS',
    'Región de Antofagasta': 'ANTOFAGASTA',
    'Región de la Araucanía': 'ARAUCANÍA',
    'Región del Libertador General Bernardo OHiggins': 'OHIGGINS',
    'Región de Aysén del General Carlos Ibáñez del Campo': 'AYSÉN',
    'Región del Ñuble': 'ÑUBLE',
    'Región del Maule': 'MAULE',
    'Región de Atacama': 'ATACAMA',
    'Región de Coquimbo': 'COQUIMBO',
    'Región de Magallanes y la Antártica Chilena': 'MAGALLANES'
}

In [27]:
df['seremi']= df['seremi'].fillna(df['region'].map(region_to_seremi))

**'cie_10_diagnostico'**: Corregir literales 'DESCONOCIDO', 'desconocido' y nulos por 'Desconocido'

In [30]:
df['cie_10_diagnostico'] = df['cie_10_diagnostico'].replace(['desconocido', 'DESCONOCIDO'], 'Desconocido').fillna('Desconocido')

**'diagnostico'**: valores nulos o desconocidos donde el valor de 'cie_10_diagnostico' es 'Desconocido'

In [39]:
df[(df['cie_10_diagnostico']=='Desconocido') & (df['diagnostico'].isnull())]

,etapa_clinica,region,seremi,nacionalidad,cie_10_diagnostico,diagnostico,Autóctono,anho_notificacion,ENO,pueblo_indigena,...,genotipo_1,recuento_linfocitos,clasificacion_final_vih,etapa_vih_sida,hospitalizacion,nombre_primer_sintoma,SeroGrupo,sexo,grupo_edad,codigo_comuna_residencia
62268,CONFIRMADA,Región de Arica y Parinacota,ARICA Y PARINACOTA,Chile,Desconocido,NaN,NaN,2020,Hepatitis A,No pertenece,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mujer,70-74,*****
62269,CONFIRMADA,Región Metropolitana de Santiago,RM,Chile,Desconocido,NaN,NaN,2020,Hepatitis A,No pertenece,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mujer,70-74,*****
62270,CONFIRMADA,Región de Antofagasta,ANTOFAGASTA,Chile,Desconocido,NaN,NaN,2020,Hepatitis A,No pertenece,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hombre,65-69,*****
62271,CONFIRMADA,Región de Valparaíso,VALPARAÍSO,Chile,Desconocido,NaN,NaN,2020,Hepatitis A,No pertenece,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mujer,70-74,*****
62274,CONFIRMADA,Región de Tarapacá,TARAPACÁ,Chile,Desconocido,NaN,NaN,2020,Hepatitis A,No pertenece,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hombre,35-39,*****
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272081,CONFIRMADA,Región del Ñuble,ÑUBLE,Chile,Desconocido,NaN,NaN,2022,Meningitis Bacteriana,No pertenece,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hombre,00-04,16101
272083,CONFIRMADA,Región Metropolitana de Santiago,RM,Chile,Desconocido,NaN,NaN,2022,Meningitis Bacteriana,No pertenece,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mujer,00-04,13401
272085,CONFIRMADA,Región del Biobío,BIOBÍO,Chile,Desconocido,NaN,NaN,2022,Meningitis Bacteriana,No pertenece,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hombre,65-69,08102
272099,CONFIRMADA,Región de Los Lagos,LOS LAGOS,Chile,Desconocido,NaN,NaN,2022,Meningitis Bacteriana,No pertenece,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hombre,70-74,10102


In [41]:
df['diagnostico']= df['diagnostico'].fillna('Desconocido')

**'nombre_instruccion'**: cambiar valores 'NO RESPONDE' y 'NO RECUERDA' a 'Desconocido'

In [47]:
df['nombre_instruccion'] = df['nombre_instruccion'].replace(['NO RESPONDE', 'NO RECUERDA'], 'Desconocido')

**'existe_coinfeccion'**

In [58]:
df['existe_coinfeccion'] = df['existe_coinfeccion'].fillna('Desconocido')

**'uso_preservativo'**

In [61]:
df['uso_preservativo'] = df['uso_preservativo'].fillna('Desconocido')

**'via_transmision'**

In [59]:
df['via_transmision'] = df['via_transmision'].replace(['DESCONOCIDO'], 'Desconocido').fillna('Desconocido')

**eliminar columnas con menos de 70000 registros**

In [53]:
df = df.loc[:, df.notnull().sum() >= 70000]

In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294352 entries, 0 to 294351
Data columns (total 17 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   etapa_clinica             294352 non-null  object
 1   region                    294352 non-null  object
 2   seremi                    294352 non-null  object
 3   nacionalidad              294352 non-null  object
 4   cie_10_diagnostico        294352 non-null  object
 5   diagnostico               294352 non-null  object
 6   anho_notificacion         294352 non-null  int64 
 7   ENO                       294352 non-null  object
 8   pueblo_indigena           294352 non-null  object
 9   nombre_instruccion        294352 non-null  object
 10  existe_coinfeccion        103633 non-null  object
 11  via_transmision           90762 non-null   object
 12  uso_preservativo          75130 non-null   object
 13  pais_contagio             294352 non-null  object
 14  sexo